In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Circle
import numpy as np
import os
import sys

In [ ]:
sys.path.append('../src/')

In [ ]:
from modules import helpers as hp
from modules import mammoscan as ms

In [ ]:
plt.style.use('dark_background') # comment or delete it for light background

In [ ]:
mammo = pd.read_table('../scan_file_data.txt', delimiter='\s', engine='python')

In [ ]:
# rename the class column to avoid conflicts with the class keyword in python
mammo.columns = ['refnum', 'bg', 'ab_class', 'severity', 'x', 'y', 'radius']

In [ ]:
mammo.ab_class.value_counts().plot(kind='bar', color='orange')

In [ ]:
mammo.severity.value_counts().plot(kind='bar', color='orange')

In [ ]:
mammo.bg.value_counts().plot(kind='bar', color='orange')

In [ ]:
mammo.info()

In [ ]:
mx = mammo.groupby(['ab_class', 'severity']).size().unstack().plot(kind='bar', 
                                                                   figsize=(10, 8), 
                                                                   xlabel='Abnormality Class', 
                                                                   ylabel='SCANS',
                                                                   title='SEVERITY BY CLASS')


In [ ]:
ab_sev = mammo.groupby(['ab_class', 'severity']).size()

In [ ]:
ab_sev = mammo.groupby('ab_class').severity.value_counts(normalize=True)
ab_sev

In [ ]:
ax = ab_sev.unstack().plot(kind='bar', 
                      stacked=True, 
                      figsize=(10, 8), 
                      xlabel='Abnormality Class', 
                      ylabel='Percentage',
                      title='SEVERITY BY CLASS');

# manipulate
vals = ax.get_yticks()
ax.set_yticklabels([f'{x:.2%}' for x in vals]);

In [ ]:
# get indices of the norm class only
norms = lambda x: x == 'NORM'
mammo.severity = mammo.severity.fillna('A')

In [ ]:
from pathlib import Path
import re

INITIALLY REMOVE ONLY THE SCANS WITHOUT INVALID X ENTRY *NOTE

In [ ]:
test_df = mammo.copy()

In [ ]:
test_df

In [ ]:
test_df.drop_duplicates(subset='refnum', keep='first', inplace=True)

In [ ]:
test_df.set_index(keys='refnum', drop=True, inplace=True)

In [ ]:
test_df.index;

In [ ]:
clean_df = hp.clean_ds_files(test_df)

In [ ]:
clean_df.info()

In [ ]:
# save clean to file
clean_df.to_csv('test_df.csv')

In [ ]:
type(clean_df.loc['mdb005'])

In [ ]:
scan_info = clean_df.loc['mdb015'].copy() # need to ensure we get a deep copy of the Series to avoid changing a value in it and keeping the reference
scan_info

In [ ]:
from PIL import Image

In [ ]:
test_img = Image.open('../all-mias/mdb015.pgm')

In [ ]:
test = ms.MammoScan(test_img, scan_info)

In [ ]:
test.pixel_matrix.shape

In [ ]:
test.plot()

In [ ]:
transformed = test.transformations
transformed

In [ ]:
for angle, imgs in transformed.items():
    for transf, img in imgs.items():
        print(f'{angle}, {transf}')
        print(np.asarray(img))
        display(img)
        
        
        
# 90 degrees left-right == 270 degree top-bottom
# 0 degrees left-right == 180 top-bottom
# drop duplicates???

In [ ]:
scans_filenames_dic = hp.create_scan_filenames_dic('../all-mias/')

In [ ]:
test_sub = hp.save_subsamples(scans_filenames_dic, clean_df)

In [ ]:
test_sub

In [ ]:
test_sub[test_sub.ab_class == 'CIRC']

In [ ]:
# create datasets by abnormalities for model creation, training and test
def create_df_by_abnormality(abnormality: str, df: pd.DataFrame) -> pd.DataFrame:
    df_ab = df.copy()
    #print(id(df))
    #print(id(df_ab))
    try:
        if 'ab_class' in df.columns:
            df_ab = df_ab[df_ab.ab_class == abnormality][['subsample_path', 'severity']]
            df_ab.reset_index(drop=True, inplace=True)
            return df_ab
    except KeyError as ke:
        print('Column ab_class does not exist in the provided DataFrame')
        print(ke)   


In [ ]:
id(test_sub)

### Using Image Generator

In [ ]:
# create the calcifications only dataset
calcifications = create_df_by_abnormality('CALC', test_sub)
calcifications

In [ ]:
# TRAIN AND TEST DATA FOR THE CALCIFICATIONS
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_generator = ImageDataGenerator(width_shift_range=.1, 
                                    height_shift_range=.1, 
                                    rotation_range=180,
                                    zoom_range=.1,
                                    horizontal_flip=True,
                                    validation_split=.2)

In [ ]:
train_data = data_generator.flow_from_dataframe(calcifications, 
                                                x_col="subsample_path", 
                                                y_col="severity",
                                                class_mode="categorical",
                                                target_size=(48,48),
                                                subset="training",
                                                color_mode="grayscale",
                                                shuffle=True)
test_data = data_generator.flow_from_dataframe(calcifications, 
                                               x_col="subsample_path", 
                                               y_col="severity",
                                               class_mode="categorical",
                                               target_size=(48,48),
                                               subset="validation",
                                               color_mode="grayscale",
                                               shuffle=False)


In [ ]:
# create neural network
from tensorflow.keras.models import Sequential

In [ ]:
cnn_calc = Sequential()

In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

In [ ]:
# add layers
cnn_calc.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(48, 48, 1)))

In [ ]:
# Add a Pooling Layer
cnn_calc.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
cnn_calc.add(Conv2D(filters=64, kernel_size=(5,5), activation='relu'))

In [ ]:
cnn_calc.add(MaxPooling2D(pool_size=(2,2)))

#### Flattening the results

In [ ]:
cnn_calc.add(Flatten())

#### Add dense layer to reduce the number of features

In [ ]:
from tensorflow.keras.layers import Dropout


In [ ]:
cnn_calc.add(Dropout(0.5)) # to avoid overfitting , drop 50%

In [ ]:
cnn_calc.add(Dense(units=1024, activation='relu'))

#### Dense layer to produce final output

In [ ]:
cnn_calc.add(Dense(units=2, activation='softmax')) # 2 units - B/ M

In [ ]:
cnn_calc.summary()

#### Visualise the Model's Structure

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
plot_model(cnn_calc, to_file='convnet.png', show_shapes=True, show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
cnn_calc.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

### Training and Evaluating the Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
cnn_calc.fit(train_data, 
             validation_data=test_data, 
             epochs=100, 
             batch_size=32,
             callbacks=[early_stopping])

In [ ]:
calcifications2 = test_sub[test_sub.ab_class == 'CALC']
calcifications2

In [ ]:
from PIL import Image
for idx, row in calcifications2.iterrows():
    print(row)
    img = Image.open(row['subsample_path'])
    # Create a figure. Equal aspect so circles look circular
    fig, ax = plt.subplots(1)

    fig.set_size_inches(12, 10)
    ax.set_aspect('equal')

    # Show the image
    ax.imshow(img)
    ax.set_ylim(bottom=0, top=1024)

    # create a circle to patch on the image
    x = pd.to_numeric(row['x'])
    y = pd.to_numeric(row['y'])
    r = pd.to_numeric(row['radius'])
    circ = Circle((x,1024-y), r, fill=False)
    ax.add_patch(circ)
    print(x, y, r)

In [ ]:
scans_filenames_dic